<a href="https://colab.research.google.com/github/AKMADOU/GMM-Group-Project/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4 as bs
import urllib
import spacy
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import heapq
from string import punctuation

In [ ]:
train_df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [ ]:
train_df.head()

Data Columns

The "review" column contains the textual information(input features) and the "sentiment" column contains the output labels. The task of any classifier is to correctly predict the "sentiment" given any "review" or textual column. Hence we have to apply our data cleaning, transformation steps to the "review" column.

In [ ]:
## Assess the shape of the data
print("The Shape of the Dataset".format(),train_df.shape)


This is the start of the analysis phase where we will first check the amount of data present in either of the sentiments. We will follow this up with some pictorial representations related to the words and frequency mappings.

In [ ]:
good_reviews=train_df[train_df['sentiment']=='positive']['review']
bad_reviews=train_df[train_df['sentiment']=='negative']['review']
print("First 10 samples of good reviews\n".format(),good_reviews[:10])
print("First 10 samples of bad reviews\n".format(),bad_reviews[:10])

In [ ]:
#Count of good and bad reviews
count=train_df['sentiment'].value_counts()
print('Total Counts of both sets'.format(),count)

print("==============")
#Creating a function to plot the counts using matplotlib
def plot_counts(count_good,count_bad):
    plt.rcParams['figure.figsize']=(6,6)
    plt.bar(0,count_good,width=0.6,label='Positive Reviews',color='Green')
    plt.legend()
    plt.bar(2,count_bad,width=0.6,label='Negative Reviews',color='Red')
    plt.legend()
    plt.ylabel('Count of Reviews')
    plt.xlabel('Types of Reviews')
    plt.show()
    
count_good=train_df[train_df['sentiment']=='positive']
count_bad=train_df[train_df['sentiment']=='negative']
plot_counts(len(count_good),len(count_bad))

In [ ]:
len(train_df),train_df.index.shape[-1]

In [ ]:
#Analyse the count of words in each segment- both positive and negative reviews
#Function for checking word length
def cal_len(data):
    return len(data)

#Create generic plotter with Seaborn
def plot_count(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
    sns.distplot(count_zeros,ax=ax1,color='Blue')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='Red')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()    




count_good_words=count_good['review'].str.split().apply(lambda z:cal_len(z))
count_bad_words=count_bad['review'].str.split().apply(lambda z:cal_len(z))
print("Positive Review Words:" + str(count_good_words))
print("Negative Review Words:" + str(count_bad_words))
plot_count(count_good_words,count_bad_words,"Positive Review","Negative Review","Reviews Word Analysis")

In [ ]:
#Analyse Stopwords

def plot_count_1(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
    sns.distplot(count_zeros,ax=ax1,color='Blue')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='Orange')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()    


stops=set(stopwords.words('english'))
count_good_stops=count_good['review'].apply(lambda z : np.mean([len(z) for w in str(z).split()]))
count_bad_stops=count_bad['review'].apply(lambda z : np.mean([len(z) for w in str(z).split()]))
plot_count_1(count_good_stops,count_bad_stops,"Positive Reviews Stopwords","Negative Reviews Stopwords","Reviews Stopwords Analysis")

In [ ]:
## Checking number of Urls
count_good_urls=count_good['review'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
count_bad_urls=count_bad['review'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

plot_count_1(count_good_stops,count_bad_stops,"Positive Reviews URLs","Negative Reviews URLs","Reviews URLs Analysis")

In [ ]:
for i in range(1,46):
    article_text = train_df.replace(str([i]), "")
print(article_text)

In [ ]:
!pip install langdetect

In [ ]:
def clean_text(text):
    row = re.sub(r"\\", " " ,text, flags=re.DOTALL)
    row = re.sub(r"\([^()]*\)", " ", row,flags=re.DOTALL) 
    row = re.sub(r"\n", " ", row,flags=re.DOTALL)
    return row

In [ ]:
# Application de la fonction 'clean_text' à chaque ligne du Dataframe
train_df['text_clean1']=train_df['review'].map(lambda X: clean_text(X))
train_df

In [ ]:
review_lens = train_df['text_clean1'].map(lambda x: len(x.split()))
n, bins, patches = plt.hist(review_lens, 50, density=True, facecolor='g', alpha=0.75)
plt.xlabel('Smarts')
plt.ylabel('Probability')

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
nlp_fr = spacy.load('en_core_web_sm') # Pour prendre en compte le vocabulaire francais.

In [ ]:
# Definition de la fonction 'cleanup_text' pour enlever les pronoms personnels et stopwords
stopwords = stopwords.words('english')

def cleanup_text(docs, logging=False):
    
    texts = []
    doc = nlp_fr(docs)
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return " ".join(texts)

In [ ]:
train_df['text_clean']=train_df['text_clean1'].map(lambda X: cleanup_text(X))
train_df

In [ ]:
print('Reviews description with punctuation and stopwords---\n')
print(train_df['text_clean1'][0])
print('\nReviews description after removing  stopwrods---\n')
print(train_df['text_clean'][0])

In [ ]:
def Punctuation(string):
 
    # punctuation marks
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in string.lower():
        if x in punctuations:
            string = string.replace(x, "")
    print(string)
Punctuation(clean_corpus)

In [ ]:
from string import punctuation
import string

punctuations = string.punctuation
print('list of punctuations:', punctuations)

def punctuation_cleaning(article):
    return article.translate(str.maketrans('', '', punctuations))

clean_corpus = punctuation_cleaning(train_df['text_clean1'][0])
print(clean_corpus)

In [ ]:
from nltk import word_tokenize

tokens = word_tokenize(clean_corpus)
print(tokens)

In [ ]:
len(tokens)

Suppression des mots recurrents (stopwords)

Stopwords are the most common words in any natural language. For the purpose of analyzing text data and building NLP models, these stopwords might not add much value to the meaning of the document. Generally, the most common words used in a text are “the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc.

In [ ]:
from nltk.corpus import stopwords
import nltk

# recuperation of stopwords in English
sw = nltk.corpus.stopwords.words('english')

tokens_wsw = []              # initialization of a list
for word in tokens:
    if word not in sw:
        tokens_wsw.append(word)
print(tokens_wsw)

Normalisation (Lemmatization)

Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma

In [ ]:
from nltk.stem import WordNetLemmatizer
tokens_wsw=['One','reviewers','mentioned','watching','1','Oz','episode','youll','hooked','They','right','exactly','happened']
lemmatizer = WordNetLemmatizer()
lemmawords = [lemmatizer.lemmatize(word) for word in tokens_wsw]
print (lemmawords)

Observations de certains mots et leurs nombre d'occurences

A word cloud  is a visual representation of words. Cloud creators are used to highlight popular words and phrases based on frequency and relevance. They provide you with quick and simple visual insights that can lead to more in-depth analyses.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Generate word cloud
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='salmon', colormap='Pastel1',
                      collocations=False, stopwords = STOPWORDS).generate(clean_corpus)
# Plot
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
  
plt.show()

In [ ]:
from collections import Counter
Counter(lemmawords).most_common(150)

Obtention des mots distincts (vocabulaire)

In [ ]:
vocab = set(lemmawords)
print(vocab)

In [ ]:
sp = spacy.load('en_core_web_sm')
def return_NER(sentence):
    # Tokeniser la phrase
    doc = sp(sentence)
    # Retourner le texte et le label pour chaque entité
    return [(X.text, X.label_) for X in doc.ents]

return_NER(clean_corpus)